In [ ]:
import json
import numpy as np

In [ ]:
with open('./data.json') as f:
    data = json.load(f)

In [ ]:
values = np.array(data['values'], dtype='float32')
values = values.reshape((data['height'], data['width']))
values

In [ ]:
values.shape

In [ ]:
import numpy as np
from bqplot import Figure, LinearScale, Axis, ColorScale
from bqplot_image_gl import ImageGL, Contour
import ipywidgets as widgets
scale_x = LinearScale(min=0, max=1)
scale_y = LinearScale(min=0, max=1)
scales = {'x': scale_x, 'y': scale_y}
axis_x = Axis(scale=scale_x, label='x')
axis_y = Axis(scale=scale_y, label='y', orientation='vertical')
scales_image = {'x': scale_x, 'y': scale_y, 'image': ColorScale(min=np.min(values).item(), max=np.max(values).item())}


In [ ]:
figure = Figure(scales=scales, axes=[axis_x, axis_y])
image = ImageGL(image=values, scales=scales_image)
contour = Contour(image=image, level=180, scales=scales_image)
figure.marks = (image, contour)

In [ ]:
figure

In [ ]:
slider = widgets.FloatSlider(value=contour.level, min=np.min(values).item(), max=np.max(values).item())
# we link from slider to contour, not back, since we will set multiple levels later on
widgets.jsdlink((slider, 'value'), (contour, 'level'))
slider

In [ ]:
cp = widgets.ColorPicker(value='purple')
widgets.jslink((cp, 'value'), (contour, 'color'))
cp

In [ ]:
contour.color = 'purple'

# Multiple levels
If level is a list of values, multiple contours lines will be drawn

In [ ]:
contour.level = 170

In [ ]:
contour.level = [150, 180]

In [ ]:
contour.level = [120, 150, 180]

In [ ]:
contour.level = [120, 180, 150]

In [ ]:
import ipyvuetify as v
import traitlets

class ValueListTextArea(v.TextField):
    values = traitlets.Any()
    
    @traitlets.default('v_model')
    def _v_model(self):
        return ", ".join(map(str, self.values))

    @traitlets.default('label')
    def _label(self):
        return "List of values"

    @traitlets.default('placeholder')
    def _placeholder(self):
        return "Enter a comma separated list of values"

    @traitlets.default('prepend_icon')
    def _prepend_icon(self):
        return 'show_chart'

    @traitlets.observe('v_model')
    def update_custom_selection(self, change):
        self.check_values()
    
    def check_values(self):
        try:
            values = ast.literal_eval(self.v_model)
        except Exception as e:
            self.error_messages = str(e)
            return
        if not isinstance(values, tuple):  # maybe we put in a single number
            if not isinstance(values, (int, float)):
                self.error_message = "Please provide numbers"
                return
            values = [values]
        for value in values:
            if not isinstance(value, (int, float)):
                self.error_message = "Please provide numbers"
                return
        self.error_messages = None
        self.values = values
        return True
values_list = ValueListTextArea(values=[120, 150, 180])
values_list

In [ ]:
widgets.dlink((values_list, 'values'), (contour, 'level'))
figure